In [ ]:
import pandas as pd
train_original = pd.read_csv(r'../dacon/train.csv')
train_original = train_original[['Canonical_Smiles', 'Inhibition']]
train_884 = pd.read_csv(r'../processed_data\aid_884_cyp3a4.csv')
train_1851 = pd.read_csv(r'../processed_data\aid_1851_cyp3a4.csv')
train_1645841 = pd.read_csv(r'../processed_data\aid_1645841_cyp3a4.csv')
train_1671201 = pd.read_csv(r'../processed_data\aid_1671201_mean.csv')
train_1671201.drop(['Unnamed: 0'],axis=1, inplace=True)
train_chembl = pd.read_csv(r'../processed_data\chembl_final.csv')
train_chembl.drop(['Unnamed: 0'],axis=1, inplace=True)
train_coldf = pd.read_csv(r'../processed_data\col_df.csv')
train_store = pd.read_csv(r'../processed_data\testosterone_df.csv')

train_original = pd.concat([train_original, train_884, train_1851, train_1645841, train_1671201, train_chembl, train_coldf, train_store], axis=0)

train_original.reset_index(drop=True, inplace=True)

# test = pd.read_csv(r'dacon\test.csv')
# submission = pd.read_csv(f'data\sample_submission.csv')

train = train_original
df_unique = train.drop_duplicates(subset=["Canonical_Smiles", "Inhibition"], keep="first")

# 2. Canonical_Smiles 기준으로 그룹화
grouped = df_unique.groupby("Canonical_Smiles")

# 그룹별 통계 계산
stats = grouped["Inhibition"].agg(["count", "mean", "min", "max"]).reset_index()

# 중복이 아닌 샘플은 그대로 유지
not_duplicated = stats[stats["count"] == 1][["Canonical_Smiles"]]
result = df_unique[df_unique["Canonical_Smiles"].isin(not_duplicated["Canonical_Smiles"])]

# 중복 그룹만 따로 처리
duplicated_stats = stats[stats["count"] > 1]

processed_rows = []

for _, row in duplicated_stats.iterrows():
    smiles = row["Canonical_Smiles"]
    inh_min = row["min"]
    inh_max = row["max"]
    inh_mean = row["mean"]
    inh_diff = inh_max - inh_min

    if inh_diff >= 30:
        # 2-1. drop group entirely
        continue
    elif inh_mean <= 20:
        # 2-2. use min value
        inh_value = inh_min
    elif inh_mean >= 70:
        # 2-3. use max value
        inh_value = inh_max
    else:
        # 2-4. use mean value
        inh_value = inh_mean

    processed_rows.append({
        "Canonical_Smiles": smiles,
        "Inhibition": inh_value
    })

# 처리된 중복 샘플을 DataFrame으로
processed_df = pd.DataFrame(processed_rows)

# 최종 결과 결합
final_df = pd.concat([result[["Canonical_Smiles", "Inhibition"]], processed_df], ignore_index=True)

# 인덱스 리셋 (선택)
final_df = final_df.reset_index(drop=True)
train = final_df.copy()
train.info()
train.to_csv(r'../new_train.csv', index = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23713 entries, 0 to 23712
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Canonical_Smiles  23713 non-null  object 
 1   Inhibition        23713 non-null  float64
dtypes: float64(1), object(1)
memory usage: 370.6+ KB
